In [ ]:
!pip install pyodbc
!pip install sqlalchemy

In [ ]:
import pyodbc
import sqlalchemy
import pandas as pd
import urllib
from datetime import datetime as dt

# импорт библиотек

In [ ]:
class Sql:
    def __init__(self, database, server="DESKTOP-ES2QHD6"):
        self.cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                                   "Server="+server+";"
                                   "Database="+database+";"
                                   "Trusted_Connection=yes;")
        self.conn = "-- {} -- successfully".format(datetime.now()
                                                         .strftime("%d/%m/%Y"))
    def query(self, req):
        cursor = sql.cnxn.execute(req)
        row = cursor.fetchone() 
        while row: 
            print(row[0])
            row = cursor.fetchone()
            
# создание класса для подключения к бд
# и совершения запросов к ней

In [ ]:
sql = Sql("BasisProDW")
sql.conn

In [ ]:
sql.query("SELECT TABLE_NAME FROM [BasisProDW].INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")

In [ ]:
df_employee = pd.read_excel('C:/Users/raspa/Desktop/data.xlsx', sheet_name='employee')
df_customer = pd.read_excel('C:/Users/raspa/Desktop/data.xlsx', sheet_name='customer')
df_product = pd.read_excel('C:/Users/raspa/Desktop/data.xlsx', sheet_name='product')
df_order = pd.read_excel('C:/Users/raspa/Desktop/data.xlsx', sheet_name='order')

# загрузка данных из excel файлов

In [ ]:
print(df_employee.isnull().sum())
print(df_customer.isnull().sum())
print(df_product.isnull().sum())
print(df_order.isnull().sum())

# проверка столбцов на пустые значения

In [ ]:
df_customer = df_customer.rename(columns = {'iscompany': 'is_company'})

# исправляю ошибку в названии

In [ ]:
df_order.order_date = df_order.order_date.apply(lambda x: dt.date(x))
df_employee.hire_date = df_employee.hire_date.apply(lambda x: dt.date(x))

# перевод типов столбцов из timestamp в datetime

In [ ]:
df_date = pd.DataFrame(columns=['id', 'the_date', 'day_of_week', 'day_of_month', 'day_of_year', 'day_name',
                                'month_of_year', 'month_name', 'weekend', 'quarter', 'year'])

# создание таблицы для дат заказов

In [ ]:
def is_weekend(n):
    if (int(n) == 5) or (int(n) == 6):
        return 1
    return 0

# функция, которая проверяет является ли день субботой или воскресеньем

In [ ]:
def cal_quarter(n):
    if int(n) in [1, 2, 3]:
        return 1
    elif int(n) in [4, 5, 6]:
        return 2
    elif int(n) in [7, 8, 9]:
        return 3
    elif int(n) in [10, 11, 12]:
        return 4
    
# функция, которая определяет квартал которому принадлежит дата

In [ ]:
for n in range(len(df_order)):
    s = df_order.order_date[n].strftime('%w, %d, %j, %A, %m, %B, %Y')
    s = s.split(',')
    weekend = is_weekend(s[0])
    quarter = cal_quarter(s[4])
    df_date = df_date.append({'id': n+1, 'the_date': df_order.order_date[n], 'day_of_week': int(s[0])+1, 'day_of_month': int(s[1]), 
                        'day_of_year': int(s[2]), 'day_name': s[3], 'month_of_year': int(s[4]), 
                        'month_name': s[5], 'weekend': weekend, 'quarter': quarter, 'year': int(s[6])}, ignore_index=True)
    df_order.order_date[n] = n+1
df_order = df_order.rename(columns = {'order_date': 'date_id'})    
# запись данных в новый датафрейм df_date

In [249]:
quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=DESKTOP-ES2QHD6;DATABASE=BasisProDW")
engine1 = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))


In [ ]:
df_employee.to_sql("dim_employee", con = engine1, if_exists='append', index=False)

In [ ]:
df_product.to_sql("dim_product", con = engine1, if_exists='append', index=False)

In [ ]:
df_customer.to_sql("dim_customer", con = engine1, if_exists='append', index=False)

In [ ]:
df_order.to_sql("fact_sales", con = engine1, if_exists='append', index=False)

In [ ]:
df_date.to_sql("dim_date", con = engine1, if_exists='append', index=False)